# Dance Video Generator - Colab Worker

This notebook runs as a remote GPU worker for the Dance Video Generator app.

## Setup:
1. Enable GPU: Runtime → Change runtime type → GPU (T4/V100/A100)
2. Run all cells
3. Copy the ngrok URL
4. Set `COLAB_WORKER_URL` in your backend .env

## GPU Tiers:
- **Free**: T4 (16GB VRAM) - Good for solo videos
- **Colab Pro**: V100 (16GB VRAM) - Better performance
- **Colab Pro+**: A100 (40GB VRAM) - Best for groups


In [ ]:
# Install dependencies
!pip install -q fastapi uvicorn pyngrok torch torchvision diffusers transformers accelerate
!pip install -q opencv-python pillow numpy scipy scikit-image imageio imageio-ffmpeg
!pip install -q ultralytics mediapipe minio yt-dlp ffmpeg-python

print("✅ Dependencies installed")

In [ ]:
# Check GPU
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Clone model repositories
!git clone https://github.com/AliaksandrSiarohin/first-order-model.git /content/fomm
!git clone https://github.com/magic-research/magic-animate.git /content/magic-animate

import sys
sys.path.append('/content/fomm')
sys.path.append('/content/magic-animate')

print("✅ Models cloned")

In [ ]:
# FastAPI server code
from fastapi import FastAPI, BackgroundTasks
from pydantic import BaseModel
from typing import Dict, Optional
import uuid
import asyncio
from datetime import datetime

app = FastAPI(title="Dance Generator Colab Worker")

# Job storage
jobs = {}

class GenerateRequest(BaseModel):
    project_id: str
    config: Dict

@app.get("/health")
def health_check():
    """Health check endpoint"""
    return {
        "status": "healthy",
        "gpu_available": torch.cuda.is_available(),
        "timestamp": datetime.utcnow().isoformat()
    }

@app.get("/gpu_info")
def get_gpu_info():
    """Get GPU information"""
    if not torch.cuda.is_available():
        return {"error": "No GPU available"}
    
    return {
        "name": torch.cuda.get_device_name(0),
        "memory_total": torch.cuda.get_device_properties(0).total_memory / 1e9,
        "memory_allocated": torch.cuda.memory_allocated(0) / 1e9,
        "memory_reserved": torch.cuda.memory_reserved(0) / 1e9
    }

@app.post("/generate")
async def generate_video(
    request: GenerateRequest,
    background_tasks: BackgroundTasks
):
    """Start video generation job"""
    job_id = str(uuid.uuid4())
    
    jobs[job_id] = {
        "status": "queued",
        "progress": 0,
        "step": "Initializing",
        "logs": []
    }
    
    # Start background task
    background_tasks.add_task(
        process_video_generation,
        job_id,
        request.project_id,
        request.config
    )
    
    return {"job_id": job_id}

@app.get("/status/{job_id}")
def get_job_status(job_id: str):
    """Get job status"""
    if job_id not in jobs:
        return {"error": "Job not found"}
    
    return jobs[job_id]

async def process_video_generation(
    job_id: str,
    project_id: str,
    config: Dict
):
    """Process video generation (runs in background)"""
    import time
    
    try:
        # TODO: Implement actual video generation
        # For now, simulate processing
        
        jobs[job_id]["status"] = "processing"
        
        for progress in range(0, 101, 10):
            time.sleep(3)
            jobs[job_id]["progress"] = progress
            jobs[job_id]["step"] = f"Processing {progress}%"
            jobs[job_id]["logs"].append(f"Progress: {progress}%")
        
        jobs[job_id]["status"] = "completed"
        jobs[job_id]["final_video_url"] = f"minio://dance-videos/{project_id}_colab.mp4"
        
    except Exception as e:
        jobs[job_id]["status"] = "failed"
        jobs[job_id]["error"] = str(e)

print("✅ FastAPI server created")

In [ ]:
# Start ngrok tunnel
from pyngrok import ngrok

# Set your ngrok auth token (get from https://dashboard.ngrok.com/get-started/your-authtoken)
# IMPORTANT: Replace with your token!
ngrok_auth_token = "YOUR_NGROK_TOKEN_HERE"  # ⚠️ REPLACE THIS

if ngrok_auth_token != "YOUR_NGROK_TOKEN_HERE":
    ngrok.set_auth_token(ngrok_auth_token)
    
    # Start ngrok tunnel
    public_url = ngrok.connect(8000)
    print(f"\n🌐 Colab Worker URL: {public_url}")
    print(f"\n⚠️ Copy this URL and set it as COLAB_WORKER_URL in your backend .env\n")
else:
    print("⚠️ Please set your ngrok auth token above!")
    print("Get it from: https://dashboard.ngrok.com/get-started/your-authtoken")

In [ ]:
# Start FastAPI server
import nest_asyncio
nest_asyncio.apply()

import uvicorn

print("\n🚀 Starting Colab Worker Server...\n")
print("Worker is ready to receive jobs!")
print("Keep this cell running to process videos.\n")

# Run server
uvicorn.run(app, host="0.0.0.0", port=8000)